# Bài toán

> **Xử lý dữ liệu sau khi thu thập báo Việt Nam (Demo + Exercise)**

> Hoàn thành bằng cách điền vào phần #### CODE theo hướng dẫn

    ```
    #### CODE
    ####
    ```


Vấn đề:

- Dữ liệu thu thập về có thể vẫn có nhiều vấn đề do lỗi khi thu thập và lỗi của bài báo (description ngắn, content ngắn, nội dung toàn ảnh, ..).
- Đồng nhất (Normalizing) giữa các trang web với nhau đưa về một dạng dữ liệu chung.

Mục tiêu:

- Xử lý từng dữ liệu của mỗi trang web (loại bỏ các bài báo không hợp lệ, đi qua từng trường dữ liệu và xử lý cơ bản)
- Hợp nhất dữ liệu các trang web với nhau

Đầu ra:

-  Một tập file JSON chứa các bài bài báo có các trường dữ liệu (Nhìn chung giống cái đã thu thập về và bổ sung  trường `web_news`):

    - `url`: link dẫn đến bài báo
    - `title`: tiêu đề bài báo
    - `description`: tóm tắt bài báo
    - `content`: nội dung bài báo
    - `metadata`: trường dữ liệu bổ sung

        - `cat`: thể loại bài báo
        - `subcat`: thể loại con của bài báo
        - `published_date`: thời gian xuất bản
        - `author`: người viết
    - `web_news`: nguồn bài báo (vnexpress, dantri, vietnamnet)
- Ví dụ về một bài báo:

    ```
    {
        "url": "https://vnexpress.net/chinh-phu-ban-hanh-nghi-dinh-moi-ve-gia-dat-4763835.html",
        "title": "Chính phủ ban hành nghị định mới về giá đất",
        "description": "Chính phủ hôm nay ban hành Nghị định 71, trong đó quy ...",
        "content": "Nghị định này có hiệu lực khi Luật Đất đai 2024 được thi hành ...",
        "metadata": {
            "cat": "Bất động sản",
            "subcat": "Chính sách",
            "published_date": 1719575647,
            "author": "Anh Tú"
        },
        "web_news": "vnexpress"
    },
    ```

# Các bước tiến hành

1. Chuẩn bị các thư viện, dữ liệu cần thiết
2. Xử lý dữ liệu từng site một
3. Gộp thành một file

## Chuẩn bị các thư viện cần thiết

In [ ]:
import os
import json

## Chuẩn bị data cần thiết (lấy data từ mục thu thập dữ liệu)

In [ ]:
# Mount với google colab nếu data từ google drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
DATA_FOLDER = "/content/drive/MyDrive/crawl-news/data"

## Xử lý dữ liệu từng site một

Mỗi trang website chúng ta kiểm tra các trường:

- description
- author
- content
- published date

### Trang web vnexpress

In [ ]:
VNEXPRESS_DATA_FOLDER = os.path.join(DATA_FOLDER, "vnexpress")
os.listdir(VNEXPRESS_DATA_FOLDER)

['thời-sự.json',
 'thế-giới.json',
 'kinh-doanh.json',
 'khoa-học.json',
 'bất-động-sản.json',
 'giải-trí.json',
 'thể-thao.json',
 'pháp-luật.json',
 'giáo-dục.json',
 'sức-khỏe.json',
 'đời-sống.json',
 'du-lịch.json',
 'số-hóa.json',
 'xe.json']

In [ ]:
# Gộp các thể loại file và gộp thành 1 list
all_docs = []
for file_name in os.listdir(VNEXPRESS_DATA_FOLDER):
    with open(os.path.join(VNEXPRESS_DATA_FOLDER, file_name)) as fIn:
        current_docs = json.load(fIn)
    # web_news field
    for doc in current_docs:
        doc["web_news"] = "vnexpress"
    all_docs.extend(current_docs)

len(all_docs)

28

In [ ]:
# Kiểm tra một văn bản
all_docs[0]

{'url': 'https://vnexpress.net/nhung-ky-vat-cua-cha-con-liet-si-phi-cong-4771504.html',
 'title': 'Những kỷ vật của cha con liệt sĩ phi công',
 'description': 'Cha con phi công Dương Văn Thanh và Dương Lê Minh hy sinh khi bay huấn luyện, nhiều kỷ vật đã được trao tặng cho Bảo tàng Lịch sử quân sự Việt Nam.',
 'content': 'Thượng tá phi công Dương Văn Thanh nhập ngũ năm 1975. Sau khi tốt nghiệp năm 1981, anh được giữ lại và trở thành giáo viên bay thuộc Trung đoàn Không quân 910, Trường Sĩ quan Không quân, Quân chủng Phòng không - Không quân.\nNgày 29/4/2005, trong bài thực hành huấn luyện trên bầu trời vịnh Nha Trang, thượng tá Thanh cùng trung úy Đào Việt Hưng (học viên) điều khiển máy bay L-39 thì bị chết máy đột ngột. Báo cáo cho Sở Chỉ huy bay, hai phi công nhận lệnh "được phép nhảy dù thoát hiểm".\nTuy nhiên, phía dưới là khu du lịch đảo Hòn Tre, cách bờ biển Nha Trang 3 km, đang đông khách. Phi công Thanh yêu cầu học viên Hưng bung dù thoát hiểm, còn anh tận dụng những giây cuối c

#### Xử lý description

In [ ]:
# Check description
all_descriptions = []
for doc in all_docs:
    all_descriptions.append(doc["description"])

Kiểm tra vài samples

In [ ]:
all_descriptions[:10]

## Ta có thể thấy một vài description bị dính địa điểm ở trước

['Cha con phi công Dương Văn Thanh và Dương Lê Minh hy sinh khi bay huấn luyện, nhiều kỷ vật đã được trao tặng cho Bảo tàng Lịch sử quân sự Việt Nam.',
 'LÂM ĐỒNGSau một năm nâng cấp và sử dụng, đường đèo Prenn, cửa ngõ vào trung tâm Đà Lạt, xuất hiện vết nứt dài 20 m, nhà thầu nhận định do mưa lớn kéo dài.',
 'Cô dâu Việt Nam ở Singapore bị chồng ngăn vào nhà sau khi con đầy tháng và không được gặp con gần một năm qua.',
 'MỸĐám cháy lớn nhất năm nay thiêu rụi 124.000 ha rừng ở California và đang tiếp tục lan nhanh, buộc hơn 4.000 người phải sơ tán.',
 'Sau khi bị huỷ niêm yết, Chủ tịch Xây dựng Hòa Bình nói sẽ chuyển HBC lên sàn UPCoM trong tháng 8 và khẳng định thị giá cổ phiếu sẽ không bị ảnh hưởng.',
 'Chủ tịch HĐQT, nhà sáng lập Tập đoàn Hà Đô - Nguyễn Trọng Thông xin từ nhiệm vì lý do sức khỏe.',
 'Công ty điện toán lượng tử PsiQuantum hôm 25/7 thông báo hợp tác với bang Illinois để xây dựng cơ sở lượng tử rộng khoảng 28.000 m2.',
 'Nhiệt độ trung bình toàn cầu tiếp tục tăng cao

In [ ]:

# Some have locations before --> remove them

string_test_1 = "ĐÀ NẴNGTổ hợp nghỉ dưỡng và giải trí Cocobay từng được hứa hẹn quy mô tới 5 tỷ USD, 51 ha, với khoảng 10.000 phòng tiêu chuẩn 3-5 sao, đang phơi mưa nắng."
string_test_2 = '"Chị cả" loạt phim "Phép thuật" Shannen Doherty qua đời do ung thư còn Allysa Milano và Rose McGowan khởi xướng chiến dịch #MeToo chống quấy rối tình dục.'
string_test_3 = "UBND HN ra lệnh ..."

def remove_location_description(description_str):
    # Calculate the maximum isupper or isspace first
    count = 0
    is_last_space = False # Only cut when the last is not space
    for chac in description_str:
        if chac.isupper() or chac.isspace():
            count += 1
            if chac.isupper():
                is_last_space = False
            else:
                is_last_space = True
        else:
            if count >= 1: # Remain the first capitalized (or not word) character
                count -= 1
            break

    return description_str[count:] if not is_last_space else description_str

remove_location_description(string_test_1)

'Tổ hợp nghỉ dưỡng và giải trí Cocobay từng được hứa hẹn quy mô tới 5 tỷ USD, 51 ha, với khoảng 10.000 phòng tiêu chuẩn 3-5 sao, đang phơi mưa nắng.'

In [ ]:
all_descriptions = []
for doc in all_docs:
    doc["description"] = remove_location_description(doc["description"])
    all_descriptions.append(doc["description"])

In [ ]:
all_descriptions[:10]

['Cha con phi công Dương Văn Thanh và Dương Lê Minh hy sinh khi bay huấn luyện, nhiều kỷ vật đã được trao tặng cho Bảo tàng Lịch sử quân sự Việt Nam.',
 'Sau một năm nâng cấp và sử dụng, đường đèo Prenn, cửa ngõ vào trung tâm Đà Lạt, xuất hiện vết nứt dài 20 m, nhà thầu nhận định do mưa lớn kéo dài.',
 'Cô dâu Việt Nam ở Singapore bị chồng ngăn vào nhà sau khi con đầy tháng và không được gặp con gần một năm qua.',
 'Đám cháy lớn nhất năm nay thiêu rụi 124.000 ha rừng ở California và đang tiếp tục lan nhanh, buộc hơn 4.000 người phải sơ tán.',
 'Sau khi bị huỷ niêm yết, Chủ tịch Xây dựng Hòa Bình nói sẽ chuyển HBC lên sàn UPCoM trong tháng 8 và khẳng định thị giá cổ phiếu sẽ không bị ảnh hưởng.',
 'Chủ tịch HĐQT, nhà sáng lập Tập đoàn Hà Đô - Nguyễn Trọng Thông xin từ nhiệm vì lý do sức khỏe.',
 'Công ty điện toán lượng tử PsiQuantum hôm 25/7 thông báo hợp tác với bang Illinois để xây dựng cơ sở lượng tử rộng khoảng 28.000 m2.',
 'Nhiệt độ trung bình toàn cầu tiếp tục tăng cao hơn kỷ lụ

Kiểm tra bằng độ dài

In [ ]:
max([len(desc) for desc in all_descriptions])

166

In [ ]:
min([len(desc) for desc in all_descriptions])

95

In [ ]:
# Kiểm tra xem có cái nào độ dài quá ngắn, quá dài không?
for doc in all_docs:
    if len(doc["description"]) < 50:
        print(doc["description"])
    if len(doc["description"]) > 220:
        print(doc["description"])

In [ ]:
# Chỉ dữ văn bản đồ dài >= 50
all_docs_new = []
for doc in all_docs:
    if len(doc["description"]) >= 50:
        all_docs_new.append(doc)

all_docs = all_docs_new

In [ ]:
len(all_docs)

28

#### Xử lý author

In [ ]:
all_authors = []
for doc in all_docs:
    all_authors.append(doc["metadata"]["author"])

all_authors[:10]

['Sơn Hà',
 'Khánh Hương - Trường Hà',
 'Đức Trung (Theo Straits Times)',
 'Phạm Giang (Theo AFP)',
 'Tất Đạt',
 'Anh Tú',
 'Thu Thảo (Theo MIT Technology Review)',
 'An Khang (Theo NBC)',
 'Khánh Đăng (theo NY Post, Realtor)',
 'Thu Hương\nThiết kế - thi công: BUTECCO\nẢnh: Dzũng Huỳnh\nCăn hộ 72 m2 dùng kính ngăn phòng nới rộng không gian\nMảng vách kính lớn được dùng làm bức màn ngăn cách phòng khách và ngủ nhỏ, vừa lấy sáng vừa tạo sự thông thoáng. 11\nNhà 75 m2 phong cách tối giản, xây trong 1,5 tháng\nKiến trúc và nội thất công trình được thiết kế theo phong cách tối giản, ưu tiên không gian mở để tiết kiệm thời gian, chi phí thực hiện. 53\nNhà phố với các khu vườn trong vỏ bọc rỗng\nCác khu vườn được sắp đặt ở nhiều cao độ khác nhau, nhằm đem đến sự tương tác nhiều nhất với không gian sinh hoạt. 23']

In [ ]:
# Kiểm tra tác giả có độ dài >= 50
for doc in all_docs:
    if len(doc["metadata"]["author"]) >= 50:
        print(doc["metadata"]["author"])

# parse errors, --> split with the first \n

Thu Hương
Thiết kế - thi công: BUTECCO
Ảnh: Dzũng Huỳnh
Căn hộ 72 m2 dùng kính ngăn phòng nới rộng không gian
Mảng vách kính lớn được dùng làm bức màn ngăn cách phòng khách và ngủ nhỏ, vừa lấy sáng vừa tạo sự thông thoáng. 11
Nhà 75 m2 phong cách tối giản, xây trong 1,5 tháng
Kiến trúc và nội thất công trình được thiết kế theo phong cách tối giản, ưu tiên không gian mở để tiết kiệm thời gian, chi phí thực hiện. 53
Nhà phố với các khu vườn trong vỏ bọc rỗng
Các khu vườn được sắp đặt ở nhiều cao độ khác nhau, nhằm đem đến sự tương tác nhiều nhất với không gian sinh hoạt. 23


In [ ]:
# Xử lý split
for doc in all_docs:
    if len(doc["metadata"]["author"]) >= 50:
        doc["metadata"]["author"] = doc["metadata"]["author"].split("\n")[0].strip()
# parse errors, --> split with the first \n

In [ ]:
# Kiểm tra tác giả có độ dài >= 100 --> Unknown
for doc in all_docs:
    if len(doc["metadata"]["author"]) >= 100:
        doc["metadata"]["author"] = "Unknown"

#### Xử lý contents

In [ ]:
contents = []
for doc in all_docs:
    contents.append(doc["content"])

Kiểm tra độ dài contents

In [ ]:
len_contents = [len(content) for content in contents]

In [ ]:
min(len_contents), max(len_contents)

(0, 5679)

In [ ]:
# Kiểm tra tại sao lại có docs len == 0
for doc in all_docs:
    if len(doc["content"]) == 0:
        print(doc)

# --> Mấy MCQs báo

{'url': 'https://vnexpress.net/tai-sao-ung-thu-gan-du-khong-uong-ruou-bia-4774755.html', 'title': 'Tại sao ung thư gan dù không uống rượu bia?', 'description': 'Nhiều người cho rằng chỉ uống rượu mới mắc ung thư gan mà không biết rằng thói quen ăn uống, vận động hay cân nặng cũng góp phần làm tăng nguy cơ phát triển bệnh này.', 'content': '', 'metadata': {'cat': 'Các bệnh', 'subcat': 'Ung thư', 'published_date': '2024-07-28T10:00:00+07:00', 'author': 'Bảo Bảo (Theo Times of India)'}, 'web_news': 'vnexpress'}
{'url': 'https://vnexpress.net/huong-dan-dang-ky-tam-tru-online-4760385.html', 'title': 'Hướng dẫn đăng ký tạm trú online', 'description': 'Người dân có thể thực hiện 5 bước trên Cổng dịch vụ công quốc gia hoặc VNeID để thực hiện khai báo đăng ký tạm trú online.', 'content': '', 'metadata': {'cat': 'Số hóa', 'subcat': 'Kinh nghiệm', 'published_date': '2024-06-20T09:05:12+07:00', 'author': 'Bá Nam - Khương Nha'}, 'web_news': 'vnexpress'}


In [ ]:
# Kiểm tra những docs có len < 300
for doc in all_docs:
    if len(doc["content"]) > 0 and len(doc["content"]) < 300:
        print(doc)

## Non sense, advisor, videos, mcqs ...


{'url': 'https://vnexpress.net/co-nen-mua-lai-nissan-sunny-2017-4773689.html', 'title': 'Có nên mua lại Nissan Sunny 2017?', 'description': 'Tôi có người quen muốn bán xe Nissan Sunny 2017, bản Premium giá 280 triệu, xe đi khoảng 70.000 km. (Thanh Tâm)', 'content': 'Tôi thấy xe nhìn tổng thể rộng rãi, đáp ứng đúng nhu cầu gia đình 5 người. Nhưng hiện nay vẫn còn băn khoăn về phụ tùng của xe, do nghe nói đắt và khó kiếm. Rất mong các bạn có kinh nghiệm chia sẻ giúp, chân thành cảm ơn nhiều.', 'metadata': {'cat': 'Xe', 'subcat': 'Diễn đàn', 'published_date': '2024-07-25T10:41:48+07:00', 'author': 'Unknown'}, 'web_news': 'vnexpress'}


In [ ]:
## -> remove all < 300 characters

all_docs_news = []
for doc in all_docs:
    if len(doc["content"]) >= 300:
        all_docs_news.append(doc)
all_docs = all_docs_news
len(all_docs)

25

#### Xử lý publish date

Đưa publish date về dạng chuẩn timestamp

In [ ]:
# Convert all of them to timestamp
from datetime import datetime

test_string_1 = "2024-06-13T08:00:00+07:00"
test_string_2 = "2024-06-28T18:54:07+07:00"

dt = datetime.fromisoformat(test_string_2)

timestamp = dt.timestamp()
int(timestamp)

1719575647

In [ ]:
# We could use GPT3.5 for converting to timestamp
def convert2timestamp(date_string):
    dt = datetime.fromisoformat(date_string)
    return int(dt.timestamp())

for doc in all_docs:
    doc['metadata']['published_date'] = convert2timestamp(doc['metadata']['published_date'])

In [ ]:
all_docs[0], len(all_docs)

({'url': 'https://vnexpress.net/nhung-ky-vat-cua-cha-con-liet-si-phi-cong-4771504.html',
  'title': 'Những kỷ vật của cha con liệt sĩ phi công',
  'description': 'Cha con phi công Dương Văn Thanh và Dương Lê Minh hy sinh khi bay huấn luyện, nhiều kỷ vật đã được trao tặng cho Bảo tàng Lịch sử quân sự Việt Nam.',
  'content': 'Thượng tá phi công Dương Văn Thanh nhập ngũ năm 1975. Sau khi tốt nghiệp năm 1981, anh được giữ lại và trở thành giáo viên bay thuộc Trung đoàn Không quân 910, Trường Sĩ quan Không quân, Quân chủng Phòng không - Không quân.\nNgày 29/4/2005, trong bài thực hành huấn luyện trên bầu trời vịnh Nha Trang, thượng tá Thanh cùng trung úy Đào Việt Hưng (học viên) điều khiển máy bay L-39 thì bị chết máy đột ngột. Báo cáo cho Sở Chỉ huy bay, hai phi công nhận lệnh "được phép nhảy dù thoát hiểm".\nTuy nhiên, phía dưới là khu du lịch đảo Hòn Tre, cách bờ biển Nha Trang 3 km, đang đông khách. Phi công Thanh yêu cầu học viên Hưng bung dù thoát hiểm, còn anh tận dụng những giây cu

#### Lưu processed data

In [ ]:
with open(os.path.join(DATA_FOLDER, "vnexpress_processed.json"), "w") as fOut:
    json.dump(all_docs, fOut, indent=4, ensure_ascii=False)

### Trang web dantri

Similar to trang web vnexpress

In [ ]:
DANTRI_DATA_FOLDER = os.path.join(DATA_FOLDER, "dantri")
os.listdir(DANTRI_DATA_FOLDER)

['xã-hội.json',
 'xe-++.json',
 'thể-thao.json',
 'giáo-dục.json',
 'sức-mạnh-số.json',
 'kinh-doanh.json',
 'thế-giới.json',
 'bất-động-sản.json',
 'giải-trí.json',
 'việc-làm.json',
 'tình-yêu.json',
 'sức-khỏe.json',
 'an-sinh.json',
 'pháp-luật.json',
 'du-lịch.json',
 'đời-sống.json',
 'khoa-học---công-nghệ.json']

In [ ]:
# Gộp các thể loại file và gộp thành 1 list

all_docs = []
for file_name in os.listdir(DANTRI_DATA_FOLDER):
    with open(os.path.join(DANTRI_DATA_FOLDER, file_name)) as fIn:
        current_docs = json.load(fIn)
    # add web_news field
    for doc in current_docs:
        doc["web_news"] = "dantri"
    all_docs.extend(current_docs)

len(all_docs)

34

#### Xử lý description

In [ ]:
#### CODE
####

#### Xử lý author

In [ ]:
#### CODE
####

#### Xử lý contents

In [ ]:
#### CODE
####

#### Xử lý publish date

In [ ]:
#### CODE
####

#### Lưu processed data

In [ ]:
with open(os.path.join(DATA_FOLDER, "dantri_processed.json"), "w") as fOut:
    json.dump(all_docs, fOut, indent=4, ensure_ascii=False)

### Trang web vietnamnet

Similar to trang web vnexpress

In [ ]:
VIETNAMNET_DATA_FOLDER = os.path.join(DATA_FOLDER, "vietnamnet")
os.listdir(VIETNAMNET_DATA_FOLDER)

['đời-sống.json',
 'thế-giới.json',
 'giáo-dục.json',
 'sức-khỏe.json',
 'thể-thao.json',
 'du-lịch.json',
 'văn-hóa.json',
 'kinh-doanh.json',
 'thời-sự.json',
 'chính-trị.json',
 'giải-trí.json',
 'thông-tin-và-truyền-thông.json',
 'ô-tô-xe-máy.json',
 'pháp-luật.json',
 'tuần-việt-nam.json',
 'bạn-đọc.json',
 'bất-động-sản.json',
 'bảo-vệ-người-tiêu-dùng.json',
 'công-nghiệp-hỗ-trợ.json',
 'dân-tộc---tôn-giáo.json',
 'thị-trường-tiêu-dùng.json',
 'dân-tộc-thiểu-số-và-miền-núi.json',
 'nông-thôn-mới.json',
 'nội-dung-chuyên-đề.json']

In [ ]:
# Group all the file
all_docs = []
for file_name in os.listdir(VIETNAMNET_DATA_FOLDER):
    with open(os.path.join(VIETNAMNET_DATA_FOLDER, file_name)) as fIn:
        current_docs = json.load(fIn)
    for doc in current_docs:
        doc["web_news"] = "vietnamnet"
    all_docs.extend(current_docs)

len(all_docs)

48

#### Xử lý description

In [1]:
#### CODE
####

#### Xử lý author

In [ ]:
#### CODE
####

['Hà Nam',
 'Linh Trang',
 'Hoài Linh',
 'Tuấn Trần',
 'Bảo Khánh',
 'Thanh Hùng',
 'Phương Thuý',
 'An Yên',
 'Lâm Hoàng',
 'Thiên Bình']

#### Xử lý contents

In [ ]:
#### CODE
####

#### Xử lý publish date

In [ ]:
#### CODE
####

#### Lưu processed data

In [ ]:
with open(os.path.join(DATA_FOLDER, "vietnamnet_processed.json"), "w") as fOut:
    json.dump(all_docs, fOut, indent=4, ensure_ascii=False)

## Gộp dữ liệu từ một website vào thành một file

In [ ]:
# Đọc từng file một
all_json_files = ["dantri_processed.json", "vietnamnet_processed.json", "vnexpress_processed.json"]
all_docs = []

for file_name in all_json_files:
    with open(os.path.join(DATA_FOLDER, file_name)) as fOut:
        all_docs.extend(json.load(fOut))

len(all_docs)


46

In [ ]:
import random
random.seed(73) # Shuffle cho đó
random.shuffle(all_docs)

## Lưu lại vào một cái
with open(os.path.join(DATA_FOLDER, "news_final.json"), "w") as fOut:
    json.dump(all_docs, fOut, indent=4, ensure_ascii=False)